#Team Notebook - Mobility - car traffic in Hamburg
## API data: Traffic data cars (measured with infrared sensors)

---


https://metaver.de/trefferanzeige?docuuid=2936465E-C045-4F5D-8614-24C3FBB522E2



#Procedure
1. Get a dataset of all Zählstellen
2. Get the corresponding observations for each Zählstelle

#1. Get dataset of all Zählstellen


In [ ]:
import json
import pandas as pd
import requests
import numpy as np


#read the data from the web
response = requests.get("https://iot.hamburg.de/v1.1/Datastreams?$filter=properties/serviceName eq 'HH_STA_AutomatisierteVerkehrsmengenerfassung' and properties/layerName eq 'Anzahl_Kfz_Zaehlstelle_1-Tag'&$top=500")
  #--> API Status Code 200: Everything went okay
print(response.json())
  

{'value': [{'description': 'Die Anzahl der von der Infrarotkamera erfassten Kfz wird für ein 1-Tag-Intervall aufsummiert.', '@iot.id': 13600, 'name': 'Kfz-Aufkommen an Verkehrszählstelle 0314962 im 1-Tag-Intervall', 'observationType': 'http://defs.opengis.net/elda-common/ogc-def/resource?uri=http://www.opengis.net/def/ogc-om/OM_CountObservation', 'observedArea': {'type': 'Point', 'coordinates': [10.057635, 53.579578]}, 'phenomenonTime': '2020-10-31T23:00:00.000Z/2021-06-20T21:59:59.000Z', 'properties': {'metadata': 'https://registry.gdi-de.org/id/de.hh/03957ed9-5fc4-48a5-997e-e46c8c207c8f', 'layerName': 'Anzahl_Kfz_Zaehlstelle_1-Tag', 'ownerData': 'Freie und Hansestadt Hamburg', 'serviceName': 'HH_STA_AutomatisierteVerkehrsmengenerfassung', 'resultNature': 'processed', 'infoLastUpdate': '2021-01-22T14:36:35.705Z', 'mediaMonitored': 'transport', 'statisticalUnit': 'count', 'aggregateDuration': 'P1D', 'aggregateSource.Datastream@iot.id': 13598, 'aggregateSource.Datastream@iot.navigationL

API Status Codes

Status codes are returned with every request that is made to a web server. Status codes indicate information about what happened with a request. Here are some codes that are relevant to GET requests:

    200: Everything went okay, and the result has been returned (if any).
    301: The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
    400: The server thinks you made a bad request. This can happen when you don’t send along the right data, among other things.
    401: The server thinks you’re not authenticated. Many APIs require login ccredentials, so this happens when you don’t send the right credentials to access an API.
    403: The resource you’re trying to access is forbidden: you don’t have the right permissions to see it.
    404: The resource you tried to access wasn’t found on the server.
    503: The server is not ready to handle the request.

You might notice that all of the status codes that begin with a ‘4’ indicate some sort of error. The first number of status codes indicate their categorization. This is useful — you can know that if your status code starts with a ‘2’ it was successful and if it starts with a ‘4’ or ‘5’ there was an error. 

## Turn the JSON response in usable data

In [ ]:
#Extract the data from the the response object
betterdata=response.json()
  #type(betterdata)
  #--> type: dict


#only select the data, which are under the heading "value" and convert this to a pandas dataframe
test = betterdata["value"]
  #type(test)
  #--> type: List
  
firstdataframe = pd.DataFrame(test)
firstdataframe.columns

#save Zählstellen IDs in a panda series for easier use later
iotids = firstdataframe["@iot.id"]

#save Zählstellen Download-Links in a panda series for easier use later
Links = firstdataframe["Observations@iot.navigationLink"]



In [ ]:
type(betterdata)

dict

In [ ]:
df = pd.Datafram.from_dict(betterdata)

AttributeError: ignored

#2. Loop through the links of all "Zählstellen" to receive a dataset of the nr. of vehicles for every day/every "Zählstelle"

In [ ]:
#first every Zählstelle is assigned a number (Sequence) to use in the loop (e.g. 493 Zählstellen --> sequence of 0 to 492)
NoZaehlstellen = len(Links.index)

Sequence = np.array(range(0,NoZaehlstellen))

# create a list for the results
ListofAll = list()


for x in Sequence:
  #by default only the first 100 observations for this "Zählstelle" will be shown, therefore we expand the results to max 600
  Testlink = Links[x]+"?$top=600"
  #read the data from the web
  Fall = requests.get(Testlink)
    #--> API Status Code 200: Everything went okay
  #Extract the data from the the Fall-object
  FallData = Fall.json()
  #only select the data, which are under the heading "value" and convert this to a pandas dataframe
  FallDataValue = FallData["value"]
  FallDataFrame = pd.DataFrame(FallDataValue)
  #New column is created with the IDs for each Zählstelle
  FallDataFrame["ZaehlstelleID"] = iotids[x]
  #all observations from all Zählstellen are appended to the final list
  ListofAll.append(FallDataFrame)
  



In [ ]:
print(ListofAll)

[       @iot.id  ... ZaehlstelleID
0     51218835  ...         13600
1     51493102  ...         13600
2     51730740  ...         13600
3     51836486  ...         13600
4     51836603  ...         13600
..         ...  ...           ...
120  149616436  ...         13600
121  150688831  ...         13600
122  152467731  ...         13600
123  154042209  ...         13600
124  155510408  ...         13600

[125 rows x 9 columns],        @iot.id  ... ZaehlstelleID
0     51218837  ...         13596
1     51493103  ...         13596
2     51730741  ...         13596
3     51836604  ...         13596
4     51836487  ...         13596
..         ...  ...           ...
120  149616437  ...         13596
121  150688832  ...         13596
122  152467732  ...         13596
123  154042210  ...         13596
124  155510409  ...         13596

[125 rows x 9 columns],        @iot.id  ... ZaehlstelleID
0     51220438  ...         14392
1     51495503  ...         14392
2     51733078  ...         143

In [ ]:
#Convert list to one single dataframe 
DataofAll = pd.concat(ListofAll)

In [ ]:
DataofAll

,@iot.id,phenomenonTime,result,resultTime,@iot.selfLink,FeatureOfInterest@iot.navigationLink,Datastream@iot.navigationLink,MultiDatastream@iot.navigationLink,ZaehlstelleID
0,51218835.0,2021-01-21T23:00:00.000Z/2021-01-22T22:59:59.000Z,3736.0,2021-01-23T01:27:30.375Z,https://iot.hamburg.de/v1.1/Observations(51218...,https://iot.hamburg.de/v1.1/Observations(51218...,https://iot.hamburg.de/v1.1/Observations(51218...,https://iot.hamburg.de/v1.1/Observations(51218...,13600
1,51493102.0,2021-01-22T23:00:00.000Z/2021-01-23T22:59:59.000Z,3972.0,2021-01-24T01:27:55.873Z,https://iot.hamburg.de/v1.1/Observations(51493...,https://iot.hamburg.de/v1.1/Observations(51493...,https://iot.hamburg.de/v1.1/Observations(51493...,https://iot.hamburg.de/v1.1/Observations(51493...,13600
2,51730740.0,2021-01-23T23:00:00.000Z/2021-01-24T22:59:59.000Z,3008.0,2021-01-25T01:29:05.365Z,https://iot.hamburg.de/v1.1/Observations(51730...,https://iot.hamburg.de/v1.1/Observations(51730...,https://iot.hamburg.de/v1.1/Observations(51730...,https://iot.hamburg.de/v1.1/Observations(51730...,13600
3,51836486.0,2020-11-30T23:00:00.000Z/2020-12-01T22:59:59.000Z,8776.0,2021-01-25T10:13:18.976Z,https://iot.hamburg.de/v1.1/Observations(51836...,https://iot.hamburg.de/v1.1/Observations(51836...,https://iot.hamburg.de/v1.1/Observations(51836...,https://iot.hamburg.de/v1.1/Observations(51836...,13600
4,51836603.0,2020-10-31T23:00:00.000Z/2020-11-01T22:59:59.000Z,6389.0,2021-01-25T10:13:58.680Z,https://iot.hamburg.de/v1.1/Observations(51836...,https://iot.hamburg.de/v1.1/Observations(51836...,https://iot.hamburg.de/v1.1/Observations(51836...,https://iot.hamburg.de/v1.1/Observations(51836...,13600
...,...,...,...,...,...,...,...,...,...
228,149616136.0,2021-06-12T22:00:00.000Z/2021-06-13T21:59:59.000Z,6293.0,2021-06-17T12:20:20.725Z,https://iot.hamburg.de/v1.1/Observations(14961...,https://iot.hamburg.de/v1.1/Observations(14961...,https://iot.hamburg.de/v1.1/Observations(14961...,https://iot.hamburg.de/v1.1/Observations(14961...,14496
229,150696560.0,2021-06-16T22:00:00.000Z/2021-06-17T21:59:59.000Z,9746.0,2021-06-18T01:39:07.131Z,https://iot.hamburg.de/v1.1/Observations(15069...,https://iot.hamburg.de/v1.1/Observations(15069...,https://iot.hamburg.de/v1.1/Observations(15069...,https://iot.hamburg.de/v1.1/Observations(15069...,14496
230,152475591.0,2021-06-17T22:00:00.000Z/2021-06-18T21:59:59.000Z,8304.0,2021-06-19T01:39:05.035Z,https://iot.hamburg.de/v1.1/Observations(15247...,https://iot.hamburg.de/v1.1/Observations(15247...,https://iot.hamburg.de/v1.1/Observations(15247...,https://iot.hamburg.de/v1.1/Observations(15247...,14496
231,154050099.0,2021-06-18T22:00:00.000Z/2021-06-19T21:59:59.000Z,7529.0,2021-06-20T01:39:00.709Z,https://iot.hamburg.de/v1.1/Observations(15405...,https://iot.hamburg.de/v1.1/Observations(15405...,https://iot.hamburg.de/v1.1/Observations(15405...,https://iot.hamburg.de/v1.1/Observations(15405...,14496


#Idea to get the coordinates in the dataset. 
- The observations with no coordinates get replaced with [0,0] 
- Question to Philipp: How can we make sure that the coordinates are assigend to the correct Zählstellen?


In [ ]:
coordinatesextract = betterdata["value"]
type(coordinatesextract)

ListofCoordinates = list()

for y in Sequence:
  try:
    #Attention: The next step is wrong, the corresponding IDs must be somehow included, in order to link the coordinates to the correct Zählstellen
    Coordinates = coordinatesextract[y]["observedArea"]["coordinates"]
  except KeyError:
    Coordinates = [0,0]
  ListofCoordinates.append(Coordinates)

In [ ]:
print(ListofCoordinates)